# Group expenses

A [mathematical programming](https://en.wikipedia.org/wiki/Mathematical_optimization) approach for settling debts within a group.

_The code in this notebook can be executed directly from your browser [here](https://www.opvious.io/examples/retro/notebooks/?path=group-expenses.ipynb)._

In [1]:
%pip install 'opvious>=0.16.1'

## Problem formulation

We first define our model using `opvious`' [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html).

In [2]:
import opvious.modeling as om

class GroupExpenses(om.Model):
    """A mixed-integer model for settling debts within a group
    
    The solution will represent the optimal transfers between group members in order to achieve fairness: each
    member will end up having paid a total amount proportional to their involvement in the group's transactions.
    """
    
    members = om.Dimension()  # Participants
    transactions = om.Dimension()  # Expenses
    is_participating = om.Parameter.indicator(transactions, members)  # 1 if a member is involved in a transaction
    payment = om.Parameter.non_negative(transactions, members)  # Amount paid by each member per transaction
    min_transfer = om.Parameter.non_negative()  # Minimum transfer threshold
    
    def __init__(self):
        super().__init__()
        # Amount to be transferred by one group member to another to achieve fairness
        self.transfer = om.Variable.non_negative(self.members, self.members, qualifiers=['sender', 'recipient'])
        # Indicator variable representing a transfer from one member to another (1 if transfer > 0, 0 otherwise)
        self.is_transferring = om.fragments.ActivationVariable(self.transfer, upper_bound=self.total_payments())
    
    def total_payments(self):
        """Total amount of money paid, used as upper-bound for transfers"""
        return om.total(self.payment(t, m) for t, m in self.transactions * self.members)
    
    def fair_payment(self, t, m):
        """Fair payment in a transaction for a given member"""
        share = self.is_participating(t, m) / om.total(self.is_participating(t, o) for o in self.members)
        return share * om.total(self.payment(t, m) for m in self.members)
    
    @om.constraint
    def zero_sum_transfers(self):
        """After netting transfers, each member should have paid the sum of their fair payments"""
        for m in self.members:
            received = om.total(self.transfer(s, m) for s in self.members)
            sent = om.total(self.transfer(m, r) for r in self.members)
            owed = om.total(self.payment(t, m) - self.fair_payment(t, m) for t in self.transactions)
            yield received - sent == owed
            
    @om.constraint
    def transfer_meets_min(self): 
        """Each positive transfer is at least equal to the minimum threshold"""
        for s, r in self.members * self.members:
            yield self.transfer(s, r) >= self.is_transferring(s, r) * self.min_transfer()
            
    @om.objective
    def minimize_total_transferred(self):
        """First objective: minimize the total amount of money transferred between members"""
        return om.total(self.transfer(s, r) for s, r in self.members * self.members)
    
    @om.fragments.magnitude_variable(members, projection=0, lower_bound=False)
    def max_transfers_sent(self, m):
        """Number of transfers sent by each member"""
        return om.total(self.is_transferring(m, r) for r in self.members)
    
    @om.objective
    def minimize_max_transfers_sent(self):
        """Second objective: minimize the maximum number of transfers sent by any member"""
        return self.max_transfers_sent()


model = GroupExpenses()
model.definition_counts().T # Summary of the model's components

title,GroupExpenses
category,
CONSTRAINT,4
DIMENSION,2
OBJECTIVE,2
PARAMETER,3
VARIABLE,3


For the mathematically inclined, models can generate their LaTeX specification.

In [3]:
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">GroupExpenses</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^v_\mathrm{transfer[sender,recipient]}&: \tau \in \mathbb{R}_+^{M \times M} \\
  \S^v_\mathrm{isTransferring}&: \tau^\mathrm{is} \in \{0, 1\}^{M \times M} \\
  \S^c_\mathrm{isTransferringActivates}&: \forall m, m' \in M, \sum_{t \in T, m'' \in M} p_{t,m''} \tau^\mathrm{is}_{m,m'} \geq \tau_{m,m'} \\
  \S^d_\mathrm{members}&: M \\
  \S^d_\mathrm{transactions}&: T \\
  \S^p_\mathrm{isParticipating}&: p^\mathrm{is} \in \{0, 1\}^{T \times M} \\
  \S^p_\mathrm{payment}&: p \in \mathbb{R}_+^{T \times M} \\
  \S^p_\mathrm{minTransfer}&: t^\mathrm{min} \in \mathbb{R}_+ \\
  \S^c_\mathrm{zeroSumTransfers}&: \forall m \in M, \sum_{m' \in M} \tau_{m',m} - \sum_{m' \in M} \tau_{m,m'} = \sum_{t \in T} \left(p_{t,m} - \frac{p^\mathrm{is}_{t,m}}{\sum_{m' \in M} p^\mathrm{is}_{t,m'}} \sum_{m' \in M} p_{t,m'}\right) \\
  \S^c_\mathrm{transferMeetsMin}&: \forall m, m' \in M, \tau_{m,m'} \geq \tau^\mathrm{is}_{m,m'} t^\mathrm{min} \\
  \S^o_\mathrm{minimizeTotalTransferred}&: \min \sum_{m, m' \in M} \tau_{m,m'} \\
  \S^v_\mathrm{maxTransfersSent}&: \sigma^\mathrm{maxTransfers} \in \mathbb{R}_+ \\
  \S^c_\mathrm{maxTransfersSentUpperBounds}&: \forall m \in M, \sigma^\mathrm{maxTransfers} \geq \sum_{m' \in M} \tau^\mathrm{is}_{m,m'} \\
  \S^o_\mathrm{minimizeMaxTransfersSent}&: \min \sigma^\mathrm{maxTransfers} \\
\end{align*}
$$
</div>
</details>
</div>

## Application

We wrap the formulation defined above into a simple function which returns the optimal transfers given input data.

A few things to note:

+ Solves run remotely--no local solver installation required--and can be configured via `opvious` [client](https://opvious.readthedocs.io/en/stable/overview.html#creating-a-client) instances.
+ We leverage `pandas` utilities directly thanks to the SDK's native support for dataframes.
+ We specify a custom [multi-objective strategy](https://opvious.readthedocs.io/en/stable/strategies.html) to efficiently pick a robust optimal solution.

In [4]:
import opvious

_client = opvious.Client.default()  # Opvious API client

async def compute_optimal_transfers(payments, min_transfer=10, tolerance=0.1):
    """Computes optimal transfers to settle expenses fairly within a group
    
    Args:
        payments: Dataframe of payments indexed by transaction where each column is a group member.
            Each member with a non-zero payment will be considered a participant in the transaction.
        min_transfer: Minimum individual transfer amount.
        tolerance: Relative slack bound on the total amount of money transferred during settlement
            used to minimize the number of outbound transfers for any one member. For example, the
            default value of 0.1 will allow transfering up to 10% more overall.
    """
    se = payments.stack()['payment']
    res = await _client.run_solve(
        specification=model.specification(),
        parameters={
            'minTransfer': min_transfer,
            'payment': se,
            'isParticipating': (se > 0).astype(int),
        },
        strategy=opvious.SolveStrategy(  # Multi-objective strategy
            epsilon_constraints=[
                # Within tolerance of the smallest total transfer amount
                opvious.EpsilonConstraint('minimizeTotalTransferred', relative_tolerance=tolerance),
                # Using the smallest possible number of transfers per member
                opvious.EpsilonConstraint('minimizeMaxTransfersSent'),
            ],
            target='minimizeTotalTransferred',  # Final target: minimize total transfer amount
        ),
        assert_feasible=True,
    )
    return res.outputs.variable('transfer').unstack(level=1).fillna(0).round(2)

## Testing

We validate our implementation by generating some representative test data.

In [5]:
import numpy as np
import pandas as pd

_names = ["emma", "noah", "ava", "liam", "isabella", "sophia", "mason", "mia", "lucas"]

def generate_random_payments(count=25, seed=35):
    """Generates a random dataframe of non-negative payments"""
    rng = np.random.default_rng(seed)
    tuples = []
    for i in range(count):
        tid = f't{i+1:02}'
        for name in _names:
            if rng.integers(2):
                continue
            tuples.append({
                'transaction': tid, 
                'name': name, 
                'payment': round(100 * rng.random(), 2),
            })
    df = pd.DataFrame(tuples).set_index(['transaction', 'name'])
    return df.unstack().fillna(0)

payments_df = generate_random_payments()
payments_df.head()

payment                                                       
name            ava   emma isabella liam  lucas  mason    mia  noah sophia
transaction                                                               
t01            9.68  45.85     0.00  0.0   0.00  81.06   0.00  93.5  45.33
t02            0.00   0.00    97.90  0.0   2.89   0.10   0.00   0.0   0.00
t03           10.18   0.00    15.99  0.0   0.00  33.39  85.45   0.0   0.00
t04            0.00  96.55     7.79  0.0   0.00   0.00   0.00   0.0  14.50
t05           83.16   0.00     0.00  0.0  20.19   0.00  27.69  11.2  11.38

Using the default tolerance of 10%, we get the following optimals transfers:

In [6]:
await compute_optimal_transfers(payments_df)

INFO:opvious.client.handlers:Validated inputs. [parameters=451]
INFO:opvious.client.handlers:Solving problem... [columns=163, rows=180]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=23, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=107, gap=87.57%]
INFO:opvious.client.handlers:Solve in progress... [iterations=234, gap=10.13%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=156.9327857142862]
INFO:opvious.client.handlers:Solve in progress... [iterations=1256, gap=0.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=73, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=350, gap=66.67%]
INFO:opvious.client.handlers:Solve in progress... [iterations=24084, gap=50.0%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=2]
INFO:

value                                    
recipient    ava  emma isabella   liam    mia   noah
sender                                              
ava         0.00   0.0     0.00   0.00   0.00  16.85
isabella   11.07   0.0     0.00   0.00  11.75   0.00
lucas       0.00   0.0     0.00   0.00  13.28   0.00
mason       0.00   0.0    18.22  40.81   0.00   0.00
sophia      0.00  38.8     0.00   0.00  15.44   0.00

In the solution above, the total amount of money transferred comes up to ~$166 and each person sends at most 2 transfers (Isabella, Mason, and Sophia send 2; Ava, Lucas send 1).

Let's see what happens if we reduce the tolerance to 0, forcing the solution to focus on minimizing total transfer amount.

In [7]:
await compute_optimal_transfers(payments_df, tolerance=0)

INFO:opvious.client.handlers:Validated inputs. [parameters=451]
INFO:opvious.client.handlers:Solving problem... [columns=163, rows=180]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=23, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=107, gap=87.57%]
INFO:opvious.client.handlers:Solve in progress... [iterations=234, gap=10.13%]
INFO:opvious.client.handlers:Solve in progress... [iterations=1256, gap=0.0%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=156.9327857142862]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=61, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=676, gap=50.0%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=4]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious

value                                     
recipient   ava   emma isabella   liam    mia   noah
sender                                              
ava         0.0  15.79      0.0   0.00   0.00   0.00
isabella    0.0   0.00      0.0  14.60   0.00   0.00
lucas       0.0   0.00      0.0  13.28   0.00   0.00
mason       0.0   0.00     10.0  12.93  19.24  16.85
sophia     10.0  23.01      0.0   0.00  21.23   0.00

As expected the total transfer amount is lower (~$157), however our other objective has increased: Mason sends four transfers!

Let's try increasing the tolerance instead to find a solution where each person sends at most a single transfer.

In [8]:
await compute_optimal_transfers(payments_df, tolerance=0.35)

INFO:opvious.client.handlers:Validated inputs. [parameters=451]
INFO:opvious.client.handlers:Solving problem... [columns=163, rows=180]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=23, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=107, gap=87.57%]
INFO:opvious.client.handlers:Solve in progress... [iterations=234, gap=10.13%]
INFO:opvious.client.handlers:Solve in progress... [iterations=1256, gap=0.0%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=156.9327857142862]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=63, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=276, gap=50.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=7943, gap=0.0%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=1]
INFO:opv

value                                     
recipient    ava   emma isabella   liam    mia   noah
sender                                               
ava         0.00  20.58     0.00   0.00   0.00   0.00
isabella   14.79   0.00     0.00   0.00   0.00   0.00
liam        0.00  18.22     0.00   0.00   0.00   0.00
lucas       0.00   0.00     0.00   0.00   0.00  13.28
mason       0.00   0.00     0.00  59.03   0.00   0.00
mia         0.00   0.00     0.00   0.00   0.00  13.76
noah        0.00   0.00    10.19   0.00   0.00   0.00
sophia      0.00   0.00     0.00   0.00  54.24   0.00

## Next steps

This notebook showed how [Opvious](https://www.opvious.io) can be used to define and apply optimization to concrete data, highlighting a few key features along the way (declarative modeling, remote solving, multi-objective support).

Check out the [SDK's documentation](https://opvious.readthedocs.io) to learn more or give one of the following extension ideas a try!

+ The formulation above assumes that all members involved in a transaction have equal share. How can we extend it to support arbitrary shares?
+ Which other multi-objective strategies would make sense and how would their solutions compare to the one above?
+ How can we extend the model to handle prior settlements (for example if Liam had already transferred some money to Noah)?